<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_6_Viernes_y_S%C3%A1bado_04_Chat_with_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install -U duckduckgo_search
!pip install wordcloud matplotlib
!pip install flagembedding
!pip install -U spacy
!python -m spacy download es_core_news_sm
!pip install --quiet openai langchain
!pip install install langchain-chroma
!pip install pypdf
!pip install sentence-transformers
!pip install unstructured
!pip install ctransformers
!pip install -U langchain-community
!pip install newspaper3k

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from duckduckgo_search import AsyncDDGS
import asyncio
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
import spacy
from newspaper import Article

In [ ]:
def extract_article_content(row):
    url = row['url']
    body = row['body']
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text if article.text else body
    except Exception as e:
        return body

In [ ]:
# Configuraciones para la búsqueda de noticias en Aguascalientes
configuraciones = [
    {"idioma": "Español", "keywords": "México", "region": "mx-es"}
]

# Diccionario para almacenar los resultados de noticias por idioma
resultados_noticias = {"Español": []}

async def buscar_noticias(idioma, keywords, region):
    async_ddgs = AsyncDDGS()
    noticias = await async_ddgs.anews(keywords=keywords, region=region, safesearch='moderate', max_results=25)
    return idioma, noticias

async def main():
    tasks = [buscar_noticias(conf['idioma'], conf['keywords'], conf['region']) for conf in configuraciones]
    results = await asyncio.gather(*tasks)
    for idioma, noticias in results:
        resultados_noticias[idioma].extend(noticias)  # Añadir noticias a la lista correspondiente del idioma
        print(f"Resultados para {idioma}: {len(noticias)} noticias encontradas.")
await main()

In [ ]:
df_noticias = pd.DataFrame(resultados_noticias["Español"])

In [ ]:
# Crear una nueva columna en el DataFrame para almacenar el contenido del artículo
df_noticias['full_content'] = df_noticias.apply(extract_article_content, axis=1)

In [ ]:
df_noticias

In [ ]:
import re
import pickle
import requests
import zipfile
from pathlib import Path
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.vectorstores import LanceDB
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=50,
)

In [ ]:
texts = text_splitter.create_documents(df_noticias["full_content"])

In [ ]:
documents = text_splitter.split_documents(texts)

In [ ]:
from FlagEmbedding import BGEM3FlagModel
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3',model_kwargs={'device': 'cpu'},encode_kwargs=encode_kwargs)


In [ ]:
#Ejecutar solo si se sstá usando la versión T4
#!sudo apt-get update && sudo apt-get install -y cuda-drivers


In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
%xterm

In [ ]:
#curl -fsSL https://ollama.com/install.sh | sh
#ollama serve & ollama pull llama3
#ollama serve & ollama pull phi3:mini
#lo tuve que hacer 2 veces.

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3:mini")
response = llm.invoke("sabes español?")
print(response)

In [ ]:
response = llm.invoke("que es un llm")
print(response)

In [ ]:
from chromadb.config import Settings
import chromadb
from langchain.vectorstores import Chroma



In [ ]:
persist_directory="./chromadbx"

In [ ]:
vectordb = Chroma.from_documents(
    documents=documents, # text data that you want to embed and store
    embedding=embeddings, # used to convert the documents into embeddings
    persist_directory=persist_directory, # tells Chroma where to store its data
    collection_name="noticias_full" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

vectordb.persist() # will make the database save any changes to the disk

In [ ]:
len(documents)

In [ ]:
query = "Que está pasando con la ola de calor en méxico, responde en español"
vectordb.similarity_search(query,10)

In [ ]:
# Initialize an instance of the Ollama model
#llm = Ollama(model="llama3")

In [ ]:
query = "Que está pasando con la ola de calor en méxico, responde en español"
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever())
respuesta = qa.run(query)
print(respuesta)